# Implementação

In [10]:
def cost(x):
    # Função de Rosenbrock
    f = (1-x[0])**2+10*(x[1]-x[0]**2)**2
    return f
def grad(x):
    g = np.zeros((2,))
    g[0] = 2*x[0]-2-40*x[0]*(x[1]-x[0]**2)
    g[1] = 20*(x[1]-x[0]**2)
    return g
def hessian(x):
    H = np.zeros((2,2))
    H[0,0] = 2-40*(x[1]-x[0]**2)+80*x[0]**2
    H[0,1] = -40*x[0]
    H[1,0] = H[0,1]
    H[1,1] = 20
    return H

Escolha $x_0$ e as constantes $0<\eta_1\leq \eta_2<1$ e $0<\gamma_1\leq \gamma_2<1$:

In [11]:
import numpy as np
from scipy.linalg import norm

x0 = np.asarray([-1,2])

delta0 = 1.0
eps = 1e-3

eta0 = 0.1
eta1 = 0.9

gamma1 = 1.75
gamma2 = 2.0

In [12]:
from moresorensen import ms

In [13]:
def btr(x, cost, grad, hessian, delta0, eta0, eta1, eps):
    
    n = x.shape[0]

    f = cost(x)
    g = grad(x)
    H = hessian(x)
    normg = norm(g)
    
    # Modelo inicial
    m = f
    norms = 1.
    rho = 0.
    rhoeps = 1.e-18
    delta = delta0
    nb_iter = 0
    verbose = True
    
    if verbose:
        print("==============================")
        print("Ponto inicial: {}".format(x))
        print("f = {}".format(f))
        print("normg = {}".format(normg))
        print("delta = {}".format(delta))
    
    while ((normg >= eps) and (norms >= 1.0e-10) and (nb_iter <= 2000)):

        # 1. Calcular a solução do subproblema
        s = ms(g,H,delta)
        norms = norm(s)
        trial = x+s
        
        # 2. Calcular a fração rho = ared/pred
        fplus = cost(trial)
        ared = f - fplus
        
        pred = -np.dot(g,s) - 0.5*np.dot(s,np.dot(H,s))
        mplus = m - pred

        # 3. Decidir sobre o passo
        if ared < rhoeps and pred < rhoeps:
            rho = 1
            fplus = np.minimum(f,fplus)
        else:
            if pred + rhoeps < rhoeps:
                rho = 1
            else:
                rho = (ared + rhoeps) / (pred + rhoeps)

        if verbose:
            print("========= Iteração {} =========".format(nb_iter))
            print("f = {}, fplus = {}".format(f, fplus))
            print("m = {}, mplus = {}".format(m,mplus))
            print("rho = {}".format(rho))
  
        if rho >= eta0:
            # A iteração foi bem sucedida. Aceite o passo
            x = trial
            f = fplus
            g = grad(x)
            normg = norm(g)
            H = hessian(x)
            m = f
            if verbose:
                print("A iteração foi bem sucedida.")
        else:
            print("A iteração não foi bem sucedida.")
        
        # 4. Atualizar o raio da região de confiança
        if rho > eta1:
            delta = gamma2*delta
        elif rho < eta0:
            delta = delta/gamma2
       
        if verbose:
            print("x: {}".format(x))
            print("f = {}".format(f))
            print("normg = {}".format(normg))
            print("delta = {}".format(delta))
   
        nb_iter += 1
        
    if normg >= eps:
        print("+++++++++++++++++++++++++")
        print("  NO CONVERGENCE OF BTR  ")
        print("+++++++++++++++++++++++++")
    else:
        print("Convergência em {} iterações.".format(nb_iter))
        
    return x,f

In [14]:
x,f = btr(x0, cost, grad, hessian, delta0, eta0, eta1, eps)

Ponto inicial: [-1  2]
f = 14
normg = 41.182520563948
delta = 1.0
========= Iteração 0 =========
f = 14, fplus = 14.0
m = 14, mplus = 14.0
rho = 1
A iteração foi bem sucedida.
x: [-1.  2.]
f = 14.0
normg = 41.182520563948
delta = 2.0
+++++++++++++++++++++++++
  NO CONVERGENCE OF BTR  
+++++++++++++++++++++++++
